# Romaji to kana with candidates FSM

some of the codes are adopted from:
https://github.com/soimort/python-romkan/blob/master/src/romkan/common.py

In [51]:
import re

In [52]:
def pairs(arr, size=2):
    for i in range(0, len(arr)-1, size):
        yield arr[i:i+size]

In [53]:
KUNREITAB_H = """ぁ      xa      あ      a      ぃ      xi      い      i      ぅ      xu
う      u      う゛      vu      う゛ぁ      va      う゛ぃ      vi       う゛ぇ      ve
う゛ぉ      vo      ぇ      xe      え      e      ぉ      xo      お      o 
か      ka      が      ga      き      ki      きゃ      kya      きゅ      kyu 
きょ      kyo      ぎ      gi      ぎゃ      gya      ぎゅ      gyu      ぎょ      gyo 
く      ku      ぐ      gu      け      ke      げ      ge      こ      ko
ご      go 
さ      sa      ざ      za      し      si      しゃ      sya      しゅ      syu 
しょ      syo      じ      zi      じゃ      zya      じゅ      zyu      じょ      zyo 
す      su      ず      zu      せ      se      ぜ      ze      そ      so
ぞ      zo 
た      ta      だ      da      ち      ti      ちゃ      tya      ちゅ      tyu 
ちょ      tyo      ぢ      di      ぢゃ      dya      ぢゅ      dyu      ぢょ      dyo 
っ      xtu 
っう゛      vvu      っう゛ぁ      vva      っう゛ぃ      vvi 
っう゛ぇ      vve      っう゛ぉ      vvo 
っか      kka      っが      gga      っき      kki      っきゃ      kkya 
っきゅ      kkyu      っきょ      kkyo      っぎ      ggi      っぎゃ      ggya 
っぎゅ      ggyu      っぎょ      ggyo      っく      kku      っぐ      ggu 
っけ      kke      っげ      gge      っこ      kko      っご      ggo      っさ      ssa 
っざ      zza      っし      ssi      っしゃ      ssya 
っしゅ      ssyu      っしょ      ssyo 
っじ      zzi      っじゃ      zzya      っじゅ      zzyu      っじょ      zzyo 
っす      ssu      っず      zzu      っせ      sse      っぜ      zze      っそ      sso 
っぞ      zzo      った      tta      っだ      dda      っち      tti 
っちゃ      ttya      っちゅ      ttyu      っちょ      ttyo      っぢ      ddi 
っぢゃ      ddya      っぢゅ      ddyu      っぢょ      ddyo      っつ      ttu 
っづ      ddu      って      tte      っで      dde      っと      tto      っど      ddo 
っは      hha      っば      bba      っぱ      ppa      っひ      hhi 
っひゃ      hhya      っひゅ      hhyu      っひょ      hhyo      っび      bbi 
っびゃ      bbya      っびゅ      bbyu      っびょ      bbyo      っぴ      ppi 
っぴゃ      ppya      っぴゅ      ppyu      っぴょ      ppyo      っふ      hhu 
っふぁ      ffa      っふぃ      ffi      っふぇ      ffe      っふぉ      ffo 
っぶ      bbu      っぷ      ppu      っへ      hhe      っべ      bbe      っぺ    ppe
っほ      hho      っぼ      bbo      っぽ      ppo      っや      yya      っゆ      yyu 
っよ      yyo      っら      rra      っり      rri      っりゃ      rrya 
っりゅ      rryu      っりょ      rryo      っる      rru      っれ      rre 
っろ      rro 
つ      tu      づ      du      て      te      で      de      と      to
ど      do 
な      na      に      ni      にゃ      nya      にゅ      nyu      にょ      nyo 
ぬ      nu      ね      ne      の      no 
は      ha      ば      ba      ぱ      pa      ひ      hi      ひゃ      hya 
ひゅ      hyu      ひょ      hyo      び      bi      びゃ      bya      びゅ      byu 
びょ      byo      ぴ      pi      ぴゃ      pya      ぴゅ      pyu      ぴょ      pyo 
ふ      hu      ふぁ      fa      ふぃ      fi      ふぇ      fe      ふぉ      fo 
ぶ      bu      ぷ      pu      へ      he      べ      be      ぺ      pe
ほ      ho      ぼ      bo      ぽ      po 
ま      ma      み      mi      みゃ      mya      みゅ      myu      みょ      myo 
む      mu      め      me      も      mo 
ゃ      xya      や      ya      ゅ      xyu      ゆ      yu      ょ      xyo
よ      yo
ら      ra      り      ri      りゃ      rya      りゅ      ryu      りょ      ryo 
る      ru      れ      re      ろ      ro 
ゎ      xwa      わ      wa      ゐ      wi      ゑ      we
を      wo      ん      n 
ん     n'
でぃ   dyi
ー     -
ちぇ    tye
っちぇ      ttye
じぇ      zye
"""

In [54]:
HEPBURNTAB_H = """ぁ      xa      あ      a      ぃ      xi      い      i      ぅ      xu
う      u      う゛      vu      う゛ぁ      va      う゛ぃ      vi      う゛ぇ      ve
う゛ぉ      vo      ぇ      xe      え      e      ぉ      xo      お      o
か      ka      が      ga      き      ki      きゃ      kya      きゅ      kyu
きょ      kyo      ぎ      gi      ぎゃ      gya      ぎゅ      gyu      ぎょ      gyo
く      ku      ぐ      gu      け      ke      げ      ge      こ      ko
ご      go      
さ      sa      ざ      za      し      shi      しゃ      sha      しゅ      shu
しょ      sho      じ      ji      じゃ      ja      じゅ      ju      じょ      jo
す      su      ず      zu      せ      se      ぜ      ze      そ      so
ぞ      zo
た      ta      だ      da      ち      chi      ちゃ      cha      ちゅ      chu
ちょ      cho      ぢ      di      ぢゃ      dya      ぢゅ      dyu      ぢょ      dyo
っ      xtsu      
っう゛      vvu      っう゛ぁ      vva      っう゛ぃ      vvi      
っう゛ぇ      vve      っう゛ぉ      vvo      
っか      kka      っが      gga      っき      kki      っきゃ      kkya      
っきゅ      kkyu      っきょ      kkyo      っぎ      ggi      っぎゃ      ggya      
っぎゅ      ggyu      っぎょ      ggyo      っく      kku      っぐ      ggu      
っけ      kke      っげ      gge      っこ      kko      っご      ggo      っさ      ssa
っざ      zza      っし      sshi      っしゃ      ssha      
っしゅ      sshu      っしょ      ssho      
っじ      jji      っじゃ      jja      っじゅ      jju      っじょ      jjo      
っす      ssu      っず      zzu      っせ      sse      っぜ      zze      っそ      sso
っぞ      zzo      った      tta      っだ      dda      っち      cchi      
っちゃ      ccha      っちゅ      cchu      っちょ      ccho      っぢ      ddi      
っぢゃ      ddya      っぢゅ      ddyu      っぢょ      ddyo      っつ      ttsu      
っづ      ddu      って      tte      っで      dde      っと      tto      っど      ddo
っは      hha      っば      bba      っぱ      ppa      っひ      hhi      
っひゃ      hhya      っひゅ      hhyu      っひょ      hhyo      っび      bbi      
っびゃ      bbya      っびゅ      bbyu      っびょ      bbyo      っぴ      ppi      
っぴゃ      ppya      っぴゅ      ppyu      っぴょ      ppyo      っふ      ffu      
っふぁ      ffa      っふぃ      ffi      っふぇ      ffe      っふぉ      ffo      
っぶ      bbu      っぷ      ppu      っへ      hhe      っべ      bbe      っぺ      ppe
っほ      hho      っぼ      bbo      っぽ      ppo      っや      yya      っゆ      yyu
っよ      yyo      っら      rra      っり      rri      っりゃ      rrya      
っりゅ      rryu      っりょ      rryo      っる      rru      っれ      rre      
っろ      rro      
つ      tsu      づ      du      て      te      で      de      と      to
ど      do      
な      na      に      ni      にゃ      nya      にゅ      nyu      にょ      nyo
ぬ      nu      ね      ne      の      no      
は      ha      ば      ba      ぱ      pa      ひ      hi      ひゃ      hya
ひゅ      hyu      ひょ      hyo      び      bi      びゃ      bya      びゅ      byu
びょ      byo      ぴ      pi      ぴゃ      pya      ぴゅ      pyu      ぴょ      pyo
ふ      fu      ふぁ      fa      ふぃ      fi      ふぇ      fe      ふぉ      fo
ぶ      bu      ぷ      pu      へ      he      べ      be      ぺ      pe
ほ      ho      ぼ      bo      ぽ      po      
ま      ma      み      mi      みゃ      mya      みゅ      myu      みょ      myo
む      mu      め      me      も      mo
ゃ      xya      や      ya      ゅ      xyu      ゆ      yu      ょ      xyo
よ      yo      
ら      ra      り      ri      りゃ      rya      りゅ      ryu      りょ      ryo
る      ru      れ      re      ろ      ro      
ゎ      xwa      わ      wa      ゐ      wi      ゑ      we
を      wo      ん      n      
ん     n'
でぃ   dyi
ー     -
ちぇ    che
っちぇ      cche
じぇ      je
"""

In [55]:
KANROM_H = {}
ROMKAN_H = {}

for pair in pairs(re.split("\s+", KUNREITAB_H + HEPBURNTAB_H)):
    kana, roma = pair
    KANROM_H[kana] = roma
    ROMKAN_H[roma] = kana

ROMKAN_H.update( {"du": "づ", "di": "ぢ", "fu": "ふ", "ti": "ち",
                "wi": "うぃ", "we": "うぇ", "wo": "を" } )


In [138]:
def create_fsm():
  fsm = {}
  for k, v in ROMKAN_H.items():
    cur = fsm
    l = len(k)
    for i, c in enumerate(k):
      if i == l - 1: # last depth
        if c in cur:
          # oops deeper entries found!
          cur[c][''] = v
        else:
          cur[c] = {'': v}
      else:
        if c not in cur:
          cur[c] = {}
        cur = cur[c]
  return fsm

fsm = {} # fsm :: map<char, fsm | char>
fsm = create_fsm()

In [139]:
fsm

{'x': {'a': {'': 'ぁ'},
  'i': {'': 'ぃ'},
  'u': {'': 'ぅ'},
  'e': {'': 'ぇ'},
  'o': {'': 'ぉ'},
  't': {'u': {'': 'っ'}, 's': {'u': {'': 'っ'}}},
  'y': {'a': {'': 'ゃ'}, 'u': {'': 'ゅ'}, 'o': {'': 'ょ'}},
  'w': {'a': {'': 'ゎ'}}},
 'a': {'': 'あ'},
 'i': {'': 'い'},
 'u': {'': 'う'},
 'v': {'u': {'': 'う゛'},
  'a': {'': 'う゛ぁ'},
  'i': {'': 'う゛ぃ'},
  'e': {'': 'う゛ぇ'},
  'o': {'': 'う゛ぉ'},
  'v': {'u': {'': 'っう゛'},
   'a': {'': 'っう゛ぁ'},
   'i': {'': 'っう゛ぃ'},
   'e': {'': 'っう゛ぇ'},
   'o': {'': 'っう゛ぉ'}}},
 'e': {'': 'え'},
 'o': {'': 'お'},
 'k': {'a': {'': 'か'},
  'i': {'': 'き'},
  'y': {'a': {'': 'きゃ'}, 'u': {'': 'きゅ'}, 'o': {'': 'きょ'}},
  'u': {'': 'く'},
  'e': {'': 'け'},
  'o': {'': 'こ'},
  'k': {'a': {'': 'っか'},
   'i': {'': 'っき'},
   'y': {'a': {'': 'っきゃ'}, 'u': {'': 'っきゅ'}, 'o': {'': 'っきょ'}},
   'u': {'': 'っく'},
   'e': {'': 'っけ'},
   'o': {'': 'っこ'}}},
 'g': {'a': {'': 'が'},
  'i': {'': 'ぎ'},
  'y': {'a': {'': 'ぎゃ'}, 'u': {'': 'ぎゅ'}, 'o': {'': 'ぎょ'}},
  'u': {'': 'ぐ'},
  'e': {'': 'げ'},
  'o':

In [67]:
def feed(node, c):
    if c not in node:
        return None
    else:
        return node[c]

def is_end(node):
    return '' in node

In [145]:
def roma2kana(s):
    ss = s.replace('nn', 'n\'')
    node = fsm
    eaten = ''
    res = ''
    for c in ss:
        cur = feed(node, c)
        if cur is None: # unaccepted
            if is_end(node): # last char was at the end node (ex. 'nk')
                res += node['']
            else:
                res += eaten
            node = feed(fsm, c)
            if node is not None:
                eaten = c
            else:
                res += c
                node = fsm
                eaten = ''
        else:
            eaten += c
            node = cur
    if is_end(node) and len(node) == 1:
        res += node['']
        node = None
        eaten = ''
    return (res, eaten)

In [152]:
roma2kana('nka1')

('んか1', '')

In [147]:
roma2kana('kpa12jj')

('kぱ12', 'jj')

In [105]:
def node2regexp(node):
    if node is None:
        return ''

    # create list of candidate kanas
    def dfs(node, lst):
        for k, v in node.items():
            if k == '':
                lst.append(v)
            else:
                dfs(v, lst)
    lst = []
    dfs(node, lst)

    # create trie of kanas
    trie = {}
    for item in lst:
        cur = trie
        for i, c in enumerate(item):
            last = i == len(item) - 1
            if last:
                cur[c] = {}
            else:
                if c in cur:
                    p = cur[c]
                    if len(p) == 0:
                        break
                    else:
                        cur = p
                else:
                    next = {}
                    cur[c] = next
                    cur = next

    # convert trie to regexp
    def to_regexp(trie):
        lst = []
        for k, v in trie.items():
            if len(v) == 0:
                lst.append(k)
            else:
                lst.append(k + to_regexp(v))
        res = '|'.join(lst)
        if len(lst) > 1:
            res = '(' + res + ')'
        return res
    return to_regexp(trie)

In [110]:
# precompute eaten (node) -> regexp
# so we need not run node2regexp every time
eaten2regexp = {'' : ''}
def dig(node, eaten):
    if '' in node and len(node) == 1:
        return # skip root node or left node
    if eaten != '':
        eaten2regexp[eaten] = node2regexp(node)
    for k, v in node.items():
        if k != '':
            dig(v, eaten + k)
dig(fsm, '')

In [111]:
eaten2regexp

{'': '',
 'x': '(ぁ|ぃ|ぅ|ぇ|ぉ|っ|ゃ|ゅ|ょ|ゎ)',
 'xt': 'っ',
 'xts': 'っ',
 'xy': '(ゃ|ゅ|ょ)',
 'xw': 'ゎ',
 'v': '(う゛|っう゛)',
 'vv': 'っう゛',
 'k': '(か|き|く|け|こ|っ(か|き|く|け|こ))',
 'ky': 'き(ゃ|ゅ|ょ)',
 'kk': 'っ(か|き|く|け|こ)',
 'kky': 'っき(ゃ|ゅ|ょ)',
 'g': '(が|ぎ|ぐ|げ|ご|っ(が|ぎ|ぐ|げ|ご))',
 'gy': 'ぎ(ゃ|ゅ|ょ)',
 'gg': 'っ(が|ぎ|ぐ|げ|ご)',
 'ggy': 'っぎ(ゃ|ゅ|ょ)',
 's': '(さ|し|す|せ|そ|っ(さ|し|す|せ|そ))',
 'sy': 'し(ゃ|ゅ|ょ)',
 'ss': 'っ(さ|し|す|せ|そ)',
 'ssy': 'っし(ゃ|ゅ|ょ)',
 'ssh': 'っし',
 'sh': 'し',
 'z': '(ざ|じ|ず|ぜ|ぞ|っ(ざ|じ|ず|ぜ|ぞ))',
 'zy': 'じ(ゃ|ゅ|ょ|ぇ)',
 'zz': 'っ(ざ|じ|ず|ぜ|ぞ)',
 'zzy': 'っじ(ゃ|ゅ|ょ)',
 't': '(た|ち|っ(た|ち|つ|て|と)|つ|て|と)',
 'ty': 'ち(ゃ|ゅ|ょ|ぇ)',
 'tt': 'っ(た|ち|つ|て|と)',
 'tty': 'っち(ゃ|ゅ|ょ|ぇ)',
 'tts': 'っつ',
 'ts': 'つ',
 'd': '(だ|ぢ|で|っ(だ|ぢ|づ|で|ど)|づ|ど)',
 'dy': '(ぢ(ゃ|ゅ|ょ)|でぃ)',
 'dd': 'っ(だ|ぢ|づ|で|ど)',
 'ddy': 'っぢ(ゃ|ゅ|ょ)',
 'h': '(っ(は|ひ|ふ|へ|ほ)|は|ひ|ふ|へ|ほ)',
 'hh': 'っ(は|ひ|ふ|へ|ほ)',
 'hhy': 'っひ(ゃ|ゅ|ょ)',
 'hy': 'ひ(ゃ|ゅ|ょ)',
 'b': '(っ(ば|び|ぶ|べ|ぼ)|ば|び|ぶ|べ|ぼ)',
 'bb': 'っ(ば|び|ぶ|べ|ぼ)',
 'bby': 'っび(ゃ|ゅ|ょ)',
 'by': 'び(ゃ|ゅ|ょ)',
 'p': '(っ(ぱ|ぴ|ぷ|ぺ|ぽ)|ぱ|ぴ|ぷ|ぺ|ぽ)',


In [134]:
reexpr.match('んか')

In [137]:
import json
json.dump(eaten2regexp, open('eaten2regex.json', 'w'), ensure_ascii=False)
json.dump(fsm, open('romaji_fsm.json', 'w'), ensure_ascii=False)

In [13]:
###
# usage
###

import json

# fsm : dict<str, (fsm | str)> (note: value is of type `str` iff  key is '')
fsm = json.load(open('src/romaji_fsm.json'))

# eaten2regexp : dict<str, regexp_str>
eaten2regexp = json.load(open('src/eaten2regex.json'))



def roma2kana(s):
    def feed(node, c):
        if c not in node:
            return None
        else:
            return node[c]

    def is_end(node):
        return '' in node
    ss = s.replace('nn', 'n\'')
    node = fsm
    eaten = ''
    res = ''
    for c in ss:
        cur = feed(node, c)
        if cur is None: # unaccepted
            if is_end(node): # last char was at the end node (ex. 'nk')
                res += node['']
            else:
                res += eaten
            node = feed(fsm, c)
            if node is not None:
                eaten = c
            else:
                res += c
                node = fsm
                eaten = ''
        else:
            eaten += c
            node = cur
    if is_end(node) and len(node) == 1:
        res += node['']
        node = None
        eaten = ''
    return (res, eaten)

def roma2expr(s):
    conv, left = roma2kana(s)
    return conv + eaten2regexp[left]

In [14]:
roma2expr('mis')

'み(さ|し|す|せ|そ|っ(さ|し|す|せ|そ))'

In [15]:
roma2expr('jisinnnokakeramo')

'じしんのかけらも'